# Support Vector Machines
In this notebook, you will classify emails as either spam or not spam using support vector machines. You are given a dataset of labeled emails, where the labels are 1 if they are ham (not spam), and -1 if they are spam. The lines of the emails have already been slightly processed, such that different words are space delimited, however little other processing has occurred. 

## Preliminary notes
1. You cannot use scikit-learn. 
2. As we move into more advanced algorithms and techniques, there will be more introductions of randomness. This means that some of the example outputs in the notebook contain some randomness, and will probably not match your results exactly. Verify your code by checking your properties/invariants or feeding in static inputs for which you can calculate the output. 

In [2]:
import gzip
import math
import numpy as np
import scipy.sparse as sp
import scipy.optimize
from collections import Counter
from testing.testing import test

In [3]:
def load_data(emails_filepath, labels_filepath):
    with gzip.open(emails_filepath, 'rt') as f:
        emails = f.readlines()
    with gzip.open(labels_filepath, 'rt') as f:
        labels = np.loadtxt(f)
    return emails, labels

Here we provide a wrapper function to cache the loaded dataset. This helps it not take quite as long to re-run. 

In [4]:
global DATA_CACHE
DATA_CACHE = None

def get_data():
    global DATA_CACHE
    if DATA_CACHE is None:
        DATA_CACHE = load_data('X1.txt.gz', 'y1.txt.gz')
    return DATA_CACHE

You will make use of the tf-idf matrix from the previous assignment. We have provided you the function below.

In [5]:
def tfidf(docs):
    all_words = set([a for a in " ".join(docs).split(" ") if a != ""])
    all_words_dict = {k:i for i,k in enumerate(all_words)}
    word_counts = [Counter([a for a in d.split(" ") if a != ""]) for d in docs]

    data = [a for wc in word_counts for a in wc.values()]
    rows = [i for i,wc in enumerate(word_counts) for a in wc.values()]
    cols = [all_words_dict[k] for wc in word_counts for k in wc.keys()]
    X = sp.coo_matrix((data, (rows,cols)), (len(docs), len(all_words)))

    idf = np.log(float(len(docs))/np.asarray((X > 0).sum(axis=0))[0])
    return X * sp.diags(idf), list(all_words)

## SVM classification
Recall the support vector machine (SVM) from slide 17 of linear classification. It a straightforward algorithm, but there are some performance- and sparsity-related caveats.

### Specification
1. If you do not use matrix operations, your code will be **very slow**. Every function in here can be implemented in 1 or 2 lines using matrix equations, and the only for loop you need is the training loop for gradient descent. **If your code is slow here, it will be extremely slow in the next section when doing parameter search**.
2. You should train your SVM using gradient descent as described in the slides. 
3. Since this is a convex function, your gradient steps should always decrease your objective. A simple check when writing these optimization procedures is to print your objectives and verify that this is the case (or plot them with matplotlib).
4. You can also use scipy.optimize.check_grad to numerically verify the correctness of your gradients.
5. For the unlikely boundary case where your hypothesis outputs 0, we will treat that as a positive prediction. 
6. Be careful of numpy.matrix objects which are constrained to always have dimension 2 (scipy operations will sometimes return this instead of an ndarray). 
7. The exact objective function to implement is: $$
\mathcal L(y, X, \theta, \lambda) = \text{sum}\left\{\text{max}\{1 - y\cdot \theta^T X, 0 \}\right\} + \frac\lambda2(\theta\cdot\theta)
$$ where $X$ is the input, $y$ is the true label, $\theta$ is the model weight, and $\lambda$ is the regularization weight. The inner `max` operation is performed over each element, and the outer `sum` operation returns the sum of the elements of the vector.
8. The gradient is:$$
    v = \mathbb1\left\{y* (X \theta)) <= 1\right\}
\\    
\\    \frac{d\theta}{d\mathcal L} = \text{sum}_\text{col}\{-y* v*X\} + \lambda\theta
$$
where the $\text{sum}_\text{col}$ operation calculates the sum over each column, producing a vector the same shape as $\theta$.

### Hints:

1. There are a few different types of multiplication involved in this: 
    - matrix-vector,
    - vector-vector dot product,
    - scalar-vector product,
    - vector-vector _element-wise_ product,
    - a vector-matrix _row-wise_ product (where each element of a vector multiplies the corresponding row of a matrix) 
2. Keep track of the shape of each intermediate product! It will help you as you implement this.
3. Read the math above and notice how we have restructured the multiplication.
4. In the gradient calculation:

    - $X$ has as many rows as the length of $y$, and as many columns as the length of $\theta$
    - $v$ is a vector containing only ones and zeros, the same size as $y$; 
    - $-y*v$ is a vector the same length as $y$
    - $(-y*v)*X$ is the same size as $X$.
5. If these equations don't make sense, you should consult the slides; the same equations are written sample-by-sample there.

Some useful tricks for debugging: 

1. Use very simple inputs (i.e. small vectors of ones) and compare the output of each function with a hand calculation. 
2. One way to guarantee your gradient is correct is to verify it numerically using a derivative approximation. You can read more about numerical differentiation methods here (https://en.wikipedia.org/wiki/Finite_difference) but for your purposes, you can use scipy.optimize.check_grad to do the numerical checking for you. 

Given the specifications, fill out the `SVM` class below.

In [208]:
def SVM_test(SVM_impl):
    # Use small random inputs and outputs for initial debugging
    y0 = np.random.randint(0, 2, 5) * 2 - 1
    X0 = np.random.random((5, 10))
    t0 = np.random.random(10)
    svm0 = SVM_impl(X0, y0, 1e-4)
    svm0.theta = t0

    def obj(theta):
        svm0.theta = theta
        return svm0.objective(svm0.X, svm0.y)

    def grad(theta):
        svm0.theta = theta
        return svm0.gradient()

    # Numerically verify the correctness of your gradients
    test.true(scipy.optimize.check_grad(obj, grad, t0) < 1e-6)
    
    # Check your gradient steps decrease your objective
    pretrain_objective = svm0.objective(svm0.X, svm0.y)
    svm0.train(verbose=True)
    train_objective = svm0.objective(svm0.X, svm0.y)
    test.true(pretrain_objective > train_objective)

    # Try training your SVM on tfidf features using the labeled email dataset
    emails, labels = get_data()
    features, all_words = tfidf(emails)
    svm = SVM_impl(features, labels, 1e-4)
    svm.train(verbose=True)
    
    # Test for perfect training classification accuracy
    yp = svm.predict(features)
    test.equal(np.mean(yp * labels < 0), 0.0)
    
    # Functionality test
    test.true(np.allclose(svm.theta[::4096], [-9.11960, -9.11960, -200.5699, -9.11960, -2156.46, -37.6301, -9.11960, -16.86656, -84.3328, -24.09544, -9.11960, -9.11960, 72.967, -16.86656, -9.11960, -9.11960, -9.11960, -9.11960, -62.4962, -9.11960, -9.11960, -112.8903, -9.11960, -9.11960, 15.05430, 18.2401, -9.11960, 325.1182, 33.7348, -16.86656, -16.86656, -9.11960, -30.98786, -9.11960, -9.11960, -9.11960, -9.11960, -16.86656, 18.2401, 18.2401, -24.09544, 27.3615, -16.86656, -45.1561, -24.09544, 18.2401, 48.4848, -9.11960, -9.11960, -56.4851, -44.07297, 18.2401, 136.8898, -24.09544, -9.11960, 75.2639, 36.4838, -250.8600, -9.11960, -9.11960, -16.8666], rtol=1e-4, atol=1e-5))

#     # Uncomment the line below to see how long your training takes to run. 
#     # Our solution takes ~1.1 seconds for 100 iterations
#     %timeit svm.train()


@test
class SVM:
    def __init__(self, X, y, reg):
        """ Initialize the SVM attributes and initialize the weights vector to the zero vector. 
            Attributes: 
                X (array_like) : training data intputs
                y (vector) : 1D numpy array of training data outputs
                reg (float) : regularizer parameter
                theta : 1D numpy array of weights
        """
        self.X = X
        self.y = y
        self.reg = reg
        self.theta = np.zeros(X.shape[1])
    
    def objective(self, X, y):
        """ Calculate the objective value of the SVM. When given the training data (self.X, self.y), this is the 
            actual objective being optimized. 
            Args:
                X (array_like) : array of examples, where each row is an example
                y (array_like) : array of outputs for the training examples
            Output:
                (float) : objective value of the SVM when calculated on X,y
        """
        
        dia = sp.diags(y)
        YX = dia @ X
        hy = YX @ self.theta

        L = np.maximum(1-hy,0).sum() + sum(self.reg*(self.theta*self.theta))/2
        return L
    
    def gradient(self):
        """ Calculate the gradient of the objective value on the training examples. 
            Output:
                (vector) : 1D numpy array containing the gradient
        """    
        m,n = self.X.shape
        dia = sp.diags(self.y)
        YX = dia @ self.X
        grad = -YX.T @ (YX @ self.theta <= 1) + (self.reg * self.theta)
        return grad
    
    def train(self, niters=100, learning_rate=1, verbose=False):
        """ Train the support vector machine with the given parameters. 
            Args: 
                niters (int) : the number of iterations of gradient descent to run
                learning_rate (float) : the learning rate (or step size) to use when training
                verbose (bool) : an optional parameter that you can use to print useful information (like objective value)
        """
        
        for i in range(niters):
            self.theta -= learning_rate * self.gradient()   
            if(verbose):
                print(self.objective(self.X, self.y))
#                 print(self.theta)
                pass

    def predict(self, X):
        """ Predict the class of each label in X. 
            Args: 
                X (array_like) : array of examples, where each row is an example
            Output:/
                (vector) : 1D numpy array containing predicted labels
        """
        return X @ self.theta >= 0

12.758221948047634
6.522213651787764
15.068218884971353
2.416479575237658
2.79608036018187
14.232263477795641
5.108294712845564
9.640214270144853
7.420274629567849
5.049311364627598
9.732020066053723
2.082145084326153
7.199322785643199
3.093528777887882
4.6672841101828935
4.246893796887925
6.979935594970222
2.393744854865954
1.0018761526834121
2.1205177486996454
0.6366026001985121
1.8474091261711612
0.2714659269526819
1.5744189380524078
0.007122571583258051
0.007121147140167117
0.007119722981950555
0.007118299108551395
0.007116875519912675
0.007115452215977449
0.007114029196688775
0.00711260646198973
0.007111184011823396
0.007109761846132871
0.007108339964861263
0.007106918367951692
0.0071054970553472855
0.007104076026991184
0.007102655282826546
0.007101234822796535
0.007099814646844323
0.007098394754913099
0.007096975146946066
0.007095555822886426
0.007094136782677409
0.007092718026262243
0.007091299553584168
0.007089881364586448
0.007088463459212344
0.007087045837405137
0.00708562849

## Model Selection: Cross validation and Parameter Grid Search 
As you may have noticed, there are parameters in the SVM learning algorithm that we chose somewhat arbitrarily: the regularization parameter and the learning rate (also technically the number of iterations for the learning algorithm, but you'll only consider the first two for simplicity). 

We were also able to achieve perfect training accuracy with these random parameters. This should make you suspicious: we have an enormous amount of features so it would be extremely easy to overfit to the data, so our model may not generalize well. 

You will now evaluate and select parameters using cross validation and grid search.

## K-fold cross validation
How can we evaluate our choice of parameters? One way is to perform k-fold cross validation, which operates as follows 

1. We split the data into k+1 randomly selected but uniformly sized pieces, and set aside one block for testing.
2. For each of the remaining k parts, we train the model on k-1 parts and validate our model on the heldout part. 
3. This gives k results, and the average of these runs gives the final result.

The idea is that by holding out part of the dataset as validation data, we can train and measure our generalization ability. Note the key fact here: the training does not see the validation data at all, which is why it measures generalization! Randomizing the groups removes bias from ordering (i.e. if these results occurred in chronological order, we don't want to train on only Monday's results to predict on Wednesday's results), and averaging over the groups reduces the variance. 

In this problem, we will use classification error rate as our result metric (so the fraction of times in which our model returns the wrong answer). Calculating this value via k-fold cross validation gives us a measure of how well our model generalizes to new data (lower error rate is better). 

### Specification
You will be filling out the `ModelSelector` class below. 
1. Implement the `__init__` function to populate the attributes `blocks` and `test_block`. Break the examples in k+1 groups as follows: 
    * break the permutation into blocks of size $\text{ceil}\left(\frac{n}{k+1}\right)$ (the last block may be shorter than the rest)
    * set aside the k+1th group as the testing block, and use the remaining k blocks for cross validation
    * use the permutation as indices to select the rows that correspond to that block
    * Example: `k=2`, `P=[1,3,2,4,5,6]` sets aside `[5,6]` as the test set, and break the remaining permutation into `[[1,3],[2,4]]` so the blocks of data for validation are `X[[1,3],:]` and `X[[2,4],:]`
    * the order of the indices in the blocks should match the order in the original permutation
2. Implement the `cross_validation` function. For each group k, train the model on all other datapoints, and compute the error rate on the held-out group. Return the average error rate over all k folds. 

Try running this on the tfidf features. Can you achieve the same performance on the validation dataset as you did on the training dataset? Remember to use a random permutation (you'll get noticeably different results).



In [211]:
def judge(a):
        if(a == True):
            return 1
        else:
            return -1
        
def ModelSelector_test(ModelSelector_impl):
    emails, labels = get_data()
    features, all_words = tfidf(emails)

    # Assign random perturbation
    P = np.random.permutation(features.shape[0])

    # Check for correct block and test block attributes
    MS = ModelSelector_impl(features, labels, P, 5, 100, SVM)
    test.equal(len(MS.blocks), 5)
    test.true(np.all(MS.blocks[0] == P[:len(MS.blocks[0])]))

    # Check cross validation error
    cv_err = MS.cross_validation(0.5, 0.1)
    test.true(cv_err < 0.05)

    # Perform grid search on learning rates and regularization
#     lr, reg = MS.grid_search(np.array([0.001, 1., 10.]), np.array([0.0001, 0.1, 1.]))

#     # Check correct selection of learning rate and regulariation parameter
#     test.equal(lr, 1.0)
#     test.equal(reg, 0.1)

    # Cross validation/SVM errors
#     test.true(MS.cross_validation(0.001, 0.0001) - 0.00924 < 1e-2)
#     test.true(MS.cross_validation(0.001, 0.1) - 0.00924 < 1e-2)
#     test.true(MS.cross_validation(0.001, 1.0) - 0.00924 < 1e-2)
#     test.true(MS.cross_validation(1.0, 0.0001) - 0.00924 < 1e-2)
#     test.true(MS.cross_validation(1.0, 0.1) - 0.00900 < 1e-2)
#     test.true(MS.cross_validation(1.0, 1.0) - 0.79388 < 1e-2)
#     test.true(MS.cross_validation(10.0, 0.0001) - 0.00912 < 1e-2)
#     test.true(MS.cross_validation(10.0, 0.1) - 0.79388 < 1e-2)
#     test.true(MS.cross_validation(10.0, 1.0) - 0.80972 < 1e-2)

    # Check test error
    lr = 1
    reg = 0.1
    err, _ = MS.test(lr,reg)
    test.true(err < 0.05)


@test
class ModelSelector:
    """ A class that performs model selection. 
        Attributes:
            blocks (list) : list of lists of indices of each block used for k-fold cross validation, e.g. blocks[i] 
            gives the indices of the examples in the ith block 
            test_block (list) : list of indices of the test block that used only for reporting results
            
    """
    def __init__(self, X, y, P, k, niters, SVM_impl):
        """ Initialize the model selection with data and split into train/valid/test sets. Split the permutation into blocks 
            and save the block indices as an attribute to the model. 
            Args:
                X (array_like) : array of features for the datapoints
                y (vector) : 1D numpy array containing the output labels for the datapoints
                P (vector) : 1D numpy array containing a random permutation of the datapoints
                k (int) : number of folds
                niters (int) : number of iterations to train for
                SVM_impl: reference to your SVM class
        """
        self.X = X
        self.y = y
        self.P = P
        self.k = k
        self.niters = niters
        self.SVM = SVM_impl
        q = math.ceil(len(P)/(k+1))
        r = q * (k+1) - len(P)
        self.test_block = [P[i] for i in range(len(P)-(q-r),len(P))]
        P_val = P[0:len(P)-(q-r)]
        block = np.array_split(P_val, k, axis = 0)
        self.blocks = [b.tolist() for b in block]


    def cross_validation(self, lr, reg):
        """ Given the permutation P in the class, evaluate the SVM using k-fold cross validation for the given parameters 
            over the permutation
            Args: 
                lr (float) : learning rate
                reg (float) : regularizer parameter
            Output: 
                (float) : the cross validated error rate
        """
        err = 0
        for i in range(self.k):
            val_idx = self.blocks[i]
            train_idx = []
            for j in range(self.k):
                if (j != i):
                    train_idx.extend(self.blocks[j])

            svm_cv = self.SVM(self.X[train_idx], self.y[train_idx], reg)
            svm_cv.train(niters=self.niters, learning_rate=lr, verbose=False)
            predict = svm_cv.predict(self.X[val_idx])
            predict_ = [judge(p) for p in predict]
            
            diff = (predict_ - self.y[val_idx])/2
            err += sum(diff * diff)/len(val_idx)
        print(err/self.k)
            
        return err/self.k
    
    def grid_search(self, lrs, regs):
        """ Given two lists of parameters for learning rate and regularization parameter, perform a grid search using
            k-wise cross validation to select the best parameters. 
            Args:  
                lrs (list) : list of potential learning rates
                regs (list) : list of potential regularizers
            Output: 
                (lr, reg) : 2-tuple of the best found parameters
        """
        best = 100
        best_lr = 0
        best_reg = 0
        for lr in lrs:
            for reg in regs:
                err = self.cross_validation(lr, reg)
                if best > err:
                    best = err
                    best_lr = lr
                    best_reg = reg
        return (best_lr, best_reg)
    
    def test(self, lr, reg):
        """ Given parameters, calculate the error rate of the test data given the rest of the data. 
            Args: 
                lr (float) : learning rate
                reg (float) : regularizer parameter
            Output: 
                (err, svm) : tuple of the error rate of the SVM on the test data and the learned model
        """

        test_idx = self.test_block
        train_idx = []
        for i in range(self.k):
            train_idx.extend(self.blocks[i])
        svm_test = self.SVM(self.X[train_idx], self.y[train_idx], reg)
        svm_test.train(niters=self.niters, learning_rate=lr, verbose=False)
        predict = svm_test.predict(self.X[test_idx])

        predict_ = [judge(p) for p in predict]
        diff = (predict_ - self.y[test_idx])/2
        err = sum(diff * diff)/len(test_idx)
        return err, svm_test

0.010437912417516498
### TESTING ModelSelector: PASSED 4/4
###



## Grid search
Now, we have a means of evaluating our choice of parameters. We can now combine this with a grid search over parameters to determine the best combination. Given two lists of parameters, we compute the classification error using k-fold cross validation for each pair of parameters, and output the parameters that produces the best validation result. Our solution takes ~ 1 minute and 45 seconds to run a grid search on learning rates [0.1, 1, 10] and regularization [0.01, 0.1, 1, 10] with 100 iterations and k=4.

### Specification
You will implement the following functions in the `ModelSelector` class. 
1. Implement the `grid_search` function. Given two lists of parameters for the learning rate and regularization, perform a grid search using k-wise cross validation to select the best parameters, and return the best found parameters.
2. Implement the `test` function. Given the best found parameters, train a new model using all the training and validation data, and return the classification accuracy on the test data.

## Feature Compression 
While you are able to get decent results using an SVM and basic tf-idf features, there are 2 main problems here:
1. The actual dataset is 8x larger than the one that you load at the start
2. The number of features is extremely bloated and consumes a lot of space and computing power for a binary classification problem

So the above methodology would actually take a lot of time and memory to run on the full dataset. Following the example you did in the text classification notebook, we would need to save the tf-idf matrix for the entire training dataset (which is enormous), and then use that to generate features on new examples. 

One way to tackle this is to generate fewer, but effective, features. For example, instead of generating full tf-idf features for every single word in every email, we can instead try to focus on keywords that frequently only occur in spam email.

Fill in the `find_frequent_indicator_words` function below.

Specification:

  - Split each document `d` into words using `d.split()`. No further preprocessing is required.
  - Determine the frequency with which words occur in spam and ham emails. Count each appearance of each word (even multiple appearances in a document).
  - Select words that only ever appear in spam and words that only ever appear in ham
  - From these words, return sets of those that appear at least `threshold` times.

In [192]:
def find_frequent_indicator_words_test(find_frequent_indicator_words_impl):
    emails, labels = get_data()
    s,h = find_frequent_indicator_words_impl(emails, labels, 50)
    test.equal(len(s), 2422)
    test.equal(len(h), 290)
    test.true('advertisement' in s)
    test.true('Stephanie' in h)

@test
def find_frequent_indicator_words(docs, y, threshold):
    """ Given a list of emails and corresponding labels, return a list of frequent indicator words 
        for spam emails and for ham emails. 
        Args:  
            emails (list) : list of strings
            labels (list) : list of integers
            threshold (int): frequency threshold
        Output: 
            (spam_frequent_words, ham_frequent_words) : 2-tuple of frequent indicator words for each class
    """

    ham_counter = Counter()
    spam_counter = Counter()
    for i,d in enumerate(docs):
        words = d.split()
        if(y[i] == 1):
            ham_counter += Counter(words)
        else:
            spam_counter += Counter(words)

    ham_frequent_words = [word for word in ham_counter if word not in spam_counter and ham_counter[word] >= threshold]
    spam_frequent_words = [word for word in spam_counter if word not in ham_counter and spam_counter[word] >= threshold]
    return (spam_frequent_words, ham_frequent_words)

### TESTING find_frequent_indicator_words: PASSED 4/4
###



## Efficient Spam Detection

Your goal here is to get at least 80% accuracy on spam detection in an efficient manner. You will use the frequent indicator words implemented above and generate 2 features per email: the number of spam indicator words and the number of ham indicator words for a total of two features. This is a huge dimensionality reduction!

Given the frequent indicator words, you will first generate 2 features per email for a given training dataset. Then, you will use your ModelSelector to perform a grid search and train your SVM on the training dataset. Finally, we will evaluate your trained SVM on a separate test set, where you aim to achieve at least 80% accuracy.  

Begin by filling out the `emails2features` function below. 

In [193]:
def email2features_test(email2features_impl):
    # Get frequent indicator words on dataset we've been using so far
    emails, labels = get_data()
    frequent_indictator_words = find_frequent_indicator_words(emails, labels, 50)

    # Generate features for a different set of emails using the frequent indicator words
    emails_tr, _ = load_data('Xtr_spam.txt.gz', 'ytr_spam.txt.gz')
    features = email2features_impl(emails_tr, frequent_indictator_words)

    test.equal(features.shape, (1000, 2))
    test.equal(list(features[0]), [64., 0.])
    test.equal(list(features[888]), [0., 1])
    
@test
def email2features(emails, frequent_indicator_words):
    """ Given a list of emails, create a matrix containing features for each email.
        Args: 
            emails (list) : list of strings
            frequent_indicator_words (tuple) : tuple containing (list of spam frequent words, 
                                               list of ham frequent words)
        Output: 
            features : matrix containing features for each email where matrix shape is (num_emails, num_features)
    """
    features = np.empty((len(emails),2))

    for i,email in enumerate(emails):
        spam = 0
        ham = 0
        for word in email.split():
            if word in frequent_indicator_words[0]:
                spam += 1
            elif word in frequent_indicator_words[1]:
                ham += 1
        features[i] = np.array([spam, ham])
    return features

### TESTING email2features: PASSED 3/3
###



Now you will fill out the `efficient_spam_detection` function below. You will first use your `emails2features` function to generate features for the given set of emails. Then, you will use your ModelSelector to perform a grid search over the learning rate and regularization parameters, and train your SVM using the chosen parameters on the given training dataset. Finally, we will evaluate your trained SVM on a separate test set, where you aim to achieve at least 80% accuracy. Note that you choose the value of k for cross-validation, the set of parameters to evaluate on, as well as the number of iterations to train for. Choose these values so that you still get at least 80% accuracy but also so that you pass the time constraints of Diderot.

In [212]:
def efficient_spam_detection_test(efficient_spam_detection_impl):
    emails, labels = get_data()
    emails_tr, labels_tr = load_data('Xtr_spam.txt.gz', 'ytr_spam.txt.gz')

    frequent_indicator_words = find_frequent_indicator_words(emails, labels, 50)    
    svm = efficient_spam_detection_impl(emails_tr, labels_tr, frequent_indicator_words)
    
    # Training error
    features = email2features(emails_tr, frequent_indicator_words)
    yp = svm.predict(features)
    test.true(np.mean(yp * labels_tr < 0) <= 0.2)

    
@test
def efficient_spam_detection(emails, labels, frequent_indicator_words):
    """ Given a training dataset of emails and labels, use the given frequent indicator words to
        generate features per email. Use your ModelSelector to perform a grid search and train your SVM
        on the training dataset, and return the trained SVM. 
        Args: 
            emails (list) : list of strings
            labels (list): list of integers
            frequent_indicator_words (tuple) : tuple containing (list of spam frequent words, 
                                               list of ham frequent words)
        Output: 
            svm (SVM): SVM instance
    """
    features = email2features(emails, frequent_indicator_words)
    
    P = np.random.permutation(features.shape[0])
    MS = ModelSelector(features, labels, P, 5, 100, SVM)
    lr, reg = MS.grid_search(np.array([0.001, 1.]), np.array([0.0001, 0.1]))
    
    err, svm_final = MS.test(lr, reg)
    svm_final.train(learning_rate = lr)
    
    return svm_final


0.2407185628742515
0.2359281437125748
0.17125748502994012
0.24191616766467067
### TESTING efficient_spam_detection: PASSED 1/1
###

